# Dataset Generation for Trend Analysis

This Notebook contains the steps to create a dataset that holds the initial kochbar dataset from October 2017 and data changes that have occured over the following 6 months until March 2018. For this analysis, we will neglect the recipes that have been published later than October 2017 for simplifying the data preparation.

To create such a dataset, we first need to load all six data files and add the relevant columns to a single dataset. During this process, we will only leave the columns in the dataset, which are actually changing over time.

In [1]:
import json
import pandas as pd

with open('data/kochbar_10.json') as data_file:    
    kochbar10 = json.load(data_file)
    
date = []
for i, r in enumerate(kochbar10):
    date = kochbar10[i]['date'].split('.')
    kochbar10[i]['year'] = date[2]
    kochbar10[i]['month'] = date[1]
    kochbar10[i]['day'] = date[0]
    
kochdf = pd.DataFrame(kochbar10)
kochdf['avg_rating'] = pd.to_numeric(kochdf['avg_rating'].str.replace(',','.'))
kochdf['clicks'] = pd.to_numeric(kochdf['clicks'])
kochdf['comment_number'] = pd.to_numeric(kochdf['comment_number'])
kochdf['favorites'] = pd.to_numeric(kochdf['favorites'])
kochdf['number_votes'] = pd.to_numeric(kochdf['number_votes'])
kochdf['time_mins'] = pd.to_numeric(kochdf['time_mins'])
kochdf['date'] = pd.to_datetime(kochdf['date'], format="%d.%m.%Y")
kochdf = kochdf.drop(['avg_rating', 'calories', 'difficulty', 'ingredients', 
                      'preparation', 'price', 'subtitle', 'time_hrs', 
                      'time_mins'], axis=1).dropna()
kochdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325828 entries, 0 to 330715
Data columns (total 10 columns):
clicks            325828 non-null int64
comment_number    325828 non-null int64
date              325828 non-null datetime64[ns]
day               325828 non-null object
favorites         325828 non-null int64
month             325828 non-null object
name              325828 non-null object
number_votes      325828 non-null int64
user              325828 non-null object
year              325828 non-null object
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 27.3+ MB


This initial DataFrame holds all recipes that will be used in the following analysis. Thus, all recipes that were published after the October dataset's last (maximum) date will be filtered.
For these preprocessing steps, we can create one function that can easily be replicated for each dataset.

When joining the newer datasets with the old one, we need a unique index to find the correct data points to join.
For this purpose, the combination of the attributes name and user will be used with the assumption that one user would not publish two recipes with exactly the same name.

In [2]:
max_date = kochdf['date'].max()  # 2017-10-13
columns = ['avg_rating', 'calories', 'date', 
           'difficulty', 'ingredients', 'preparation', 
           'price', 'subtitle', 'time_hrs', 'time_mins']
numerics = ['clicks', 'comment_number', 'favorites', 'number_votes']

def preprocess(data):
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'], format="%d.%m.%Y")
    df = df.loc[df['date'] <= max_date]
    for c in numerics:
        df[c] = pd.to_numeric(df[c])
    df = df.drop(columns, axis=1).dropna()
    return df

In [3]:
with open('data/kochbar_11.json') as data_file:    
    kochbar11 = json.load(data_file)
koch11df = preprocess(kochbar11)
koch11df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325375 entries, 0 to 330561
Data columns (total 6 columns):
clicks            325375 non-null int64
comment_number    325375 non-null int64
favorites         325375 non-null int64
name              325375 non-null object
number_votes      325375 non-null int64
user              325375 non-null object
dtypes: int64(4), object(2)
memory usage: 17.4+ MB


In [4]:
kochdf = pd.merge(kochdf, koch11df,  how='left', left_on=['name', 'user'], 
                  right_on = ['name', 'user'], suffixes=('','_11'))
kochdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328282 entries, 0 to 328281
Data columns (total 14 columns):
clicks               328282 non-null int64
comment_number       328282 non-null int64
date                 328282 non-null datetime64[ns]
day                  328282 non-null object
favorites            328282 non-null int64
month                328282 non-null object
name                 328282 non-null object
number_votes         328282 non-null int64
user                 328282 non-null object
year                 328282 non-null object
clicks_11            251475 non-null float64
comment_number_11    251475 non-null float64
favorites_11         251475 non-null float64
number_votes_11      251475 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(4), object(5)
memory usage: 37.6+ MB


In [5]:
len(kochdf.loc[kochdf['date'] == max_date])

18

In [6]:
with open('data/kochbar_12.json') as data_file:    
    kochbar12 = json.load(data_file)

koch12df = preprocess(kochbar12)
koch12df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418631 entries, 0 to 425624
Data columns (total 6 columns):
clicks            418631 non-null int64
comment_number    418631 non-null int64
favorites         418631 non-null int64
name              418631 non-null object
number_votes      418631 non-null int64
user              418631 non-null object
dtypes: int64(4), object(2)
memory usage: 22.4+ MB


In [7]:
kochdf = pd.merge(kochdf, koch12df,  how='left', left_on=['name','user'], 
                  right_on = ['name','user'], suffixes=('','_12'))

In [8]:
len(kochdf.loc[kochdf['date'] == max_date])

22

In [9]:
with open('data/kochbar_01.json') as data_file:    
    kochbar01 = json.load(data_file)
    
koch01df = preprocess(kochbar01)
koch01df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414061 entries, 0 to 421349
Data columns (total 6 columns):
clicks            414061 non-null int64
comment_number    414061 non-null int64
favorites         414061 non-null int64
name              414061 non-null object
number_votes      414061 non-null int64
user              414061 non-null object
dtypes: int64(4), object(2)
memory usage: 22.1+ MB


In [10]:
kochdf = pd.merge(kochdf, koch01df,  how='left', left_on=['name','user'], 
                  right_on = ['name','user'], suffixes=('','_01'))

In [11]:
len(kochdf.loc[kochdf['date'] == max_date])

30

In [12]:
with open('data/kochbar_02.json') as data_file:    
    kochbar02 = json.load(data_file)
    
koch02df = preprocess(kochbar02)
koch02df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413039 entries, 0 to 420574
Data columns (total 6 columns):
clicks            413039 non-null int64
comment_number    413039 non-null int64
favorites         413039 non-null int64
name              413039 non-null object
number_votes      413039 non-null int64
user              413039 non-null object
dtypes: int64(4), object(2)
memory usage: 22.1+ MB


In [13]:
kochdf = pd.merge(kochdf, koch02df,  how='left', left_on=['name','user'], 
                  right_on = ['name','user'], suffixes=('','_02'))

In [14]:
len(kochdf.loc[kochdf['date'] == max_date])

38

In [15]:
with open('data/kochbar_03.json') as data_file:    
    kochbar03 = json.load(data_file)
    
koch03df = preprocess(kochbar03)
koch03df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 405977 entries, 0 to 413765
Data columns (total 6 columns):
clicks            405977 non-null int64
comment_number    405977 non-null int64
favorites         405977 non-null int64
name              405977 non-null object
number_votes      405977 non-null int64
user              405977 non-null object
dtypes: int64(4), object(2)
memory usage: 21.7+ MB


In [16]:
kochdf = pd.merge(kochdf, koch03df,  how='left', left_on=['name','user'], 
                  right_on = ['name','user'], suffixes=('','_03'))

In [17]:
len(kochdf.loc[kochdf['date'] == max_date])

54

In [18]:
# How do those 54 recipes look?
kochdf.loc[kochdf['date'] == max_date]

,clicks,comment_number,date,day,favorites,month,name,number_votes,user,year,...,favorites_01,number_votes_01,clicks_02,comment_number_02,favorites_02,number_votes_02,clicks_03,comment_number_03,favorites_03,number_votes_03
603679,21,0,2017-10-13,13,16,10,Béchamelsoße,0,Rezepte-Videos,2017,...,68.0,1.0,632.0,0.0,68.0,1.0,646.0,0.0,68.0,1.0
1117141,17,0,2017-10-13,13,11,10,Saftiger Marmorkuchen,0,Rezepte-Videos,2017,...,54.0,1.0,1441.0,0.0,54.0,1.0,1508.0,0.0,54.0,1.0
2789707,56,4,2017-10-13,13,1,10,Herzhafte Zucchini-Tomaten-Soße,9,Muehlenwirtin,2017,...,3.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2835958,37,0,2017-10-13,13,1,10,Reis: Reisbratlinge mit Champignons,2,lunapiena,2017,...,1.0,15.0,534.0,6.0,1.0,15.0,554.0,6.0,1.0,15.0
2835960,143,0,2017-10-13,13,1,10,Gefülltes Hähnchenbrustfilet im Speckmantel un...,0,Das perfekte Dinner,2017,...,2.0,0.0,924.0,0.0,3.0,0.0,NaN,NaN,NaN,NaN
2927523,80,0,2017-10-13,13,0,10,Jakobsmuscheln auf Avocado-Mango-Tatar und Zwi...,0,Das perfekte Dinner,2017,...,NaN,NaN,596.0,0.0,2.0,0.0,NaN,NaN,NaN,NaN
2927525,23,1,2017-10-13,13,0,10,Ananas-Kirsch-Teller,1,AZap,2017,...,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2927526,4,0,2017-10-13,13,0,10,Knoblauch - Zitronentrunk,0,Forelle1962,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3021329,82,2,2017-10-13,13,0,10,Apfel-Streusel-Kuchen,1,Kerstin-Susanne,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3034988,10,0,2017-10-13,13,0,10,Börek mit Gemüse - Hackfleisch - Füllung,0,Fair_Ophelia,2017,...,2.0,2.0,340.0,0.0,2.0,2.0,409.0,0.0,2.0,2.0


In [19]:
kochdf.head(10)

,clicks,comment_number,date,day,favorites,month,name,number_votes,user,year,...,favorites_01,number_votes_01,clicks_02,comment_number_02,favorites_02,number_votes_02,clicks_03,comment_number_03,favorites_03,number_votes_03
0,1572071,24,2015-07-28,28,1205,07,Japanischer Soufflé-Käsekuchen,21,KB_Redaktion,2015,...,1276.0,21.0,1611271.0,24.0,1363.0,21.0,1622373.0,25.0,1370.0,21.0
1,1147219,57,2009-02-19,19,724,02,"Chili con Carne ""Spezial""",153,mareikus,2009,...,796.0,155.0,1407336.0,58.0,824.0,155.0,1428775.0,58.0,845.0,156.0
2,3221835,47,2009-11-16,16,742,11,Pfannkuchen Grundrezept,142,zaphi67,2009,...,761.0,143.0,3745259.0,47.0,769.0,143.0,3866817.0,47.0,778.0,143.0
3,337272,77,2009-01-30,30,570,01,Allerbester Käsekuchen♥,157,Schokoprinzessin,2009,...,612.0,161.0,425456.0,77.0,629.0,161.0,452902.0,77.0,646.0,161.0
4,344673,51,2009-05-08,08,606,05,Flammkuchen Schneller Flammkuchen ohne Hefe,134,KathiB,2009,...,622.0,134.0,362521.0,51.0,627.0,134.0,365170.0,51.0,634.0,135.0
5,101973,116,2008-11-03,03,641,11,Pizza-Schnecken,151,TimejaKijara,2008,...,643.0,151.0,106629.0,116.0,644.0,151.0,108421.0,116.0,644.0,151.0
6,95075,58,2009-07-26,26,775,07,Zucchini-Hackauflauf mit Frischkäse und Spätzle,140,romantica,2009,...,848.0,141.0,99995.0,59.0,891.0,141.0,101092.0,59.0,925.0,141.0
7,620361,54,2009-06-12,12,586,06,Amerikanische Pancakes,132,Eni0,2009,...,689.0,132.0,628433.0,56.0,698.0,132.0,630445.0,56.0,735.0,132.0
8,177904,130,2009-01-30,30,529,01,Allerbester Rhabarberkuchen♥,244,Schokoprinzessin,2009,...,528.0,244.0,178564.0,130.0,529.0,244.0,178984.0,130.0,530.0,244.0
9,799637,80,2008-06-09,09,480,06,Schichtsalat,232,Kruemelmonster,2008,...,489.0,232.0,840732.0,80.0,491.0,232.0,848682.0,80.0,494.0,232.0


In [20]:
# The user "Rezeptesammlerin" seems to have many duplicate recipes
# How many recipes does she have in total?
len(kochdf.loc[kochdf['user'] == "Rezeptsammlerin"]['name'])

870

## Inconsistent data

As we can easily see from the short evaluations between the merging steps for the different data snapshots, the data on kochbar has not been handled consistently. 
With every new snapshot, there are more recipes occuring for past dates as well.
This means that in the snapshot from November 1st, kochbar.de had 22 recipes that have been published on 13.10.2017, but in the snapshot from December 1st, there were 30 recipes with the publication date 13.10.2017.
The snapshot from March 1st even shows 54 recipes from the same date, of which some are duplicates.
However, those duplicates have the same name and user, but they do not have all the same values

This is actually quite confusing and should not happen in a consistent database. Instead, this shows that kochbar is not handling their database well, because they are causing many inefficient redundancies.

To reduce the impact of this issue, we can filter duplicates. This reduces the amount of recipes from 3.3 million to 323,895.

In [21]:
kochdf = kochdf.drop_duplicates(subset=['name', 'user'])
kochdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 323895 entries, 0 to 3306825
Data columns (total 30 columns):
clicks               323895 non-null int64
comment_number       323895 non-null int64
date                 323895 non-null datetime64[ns]
day                  323895 non-null object
favorites            323895 non-null int64
month                323895 non-null object
name                 323895 non-null object
number_votes         323895 non-null int64
user                 323895 non-null object
year                 323895 non-null object
clicks_11            247174 non-null float64
comment_number_11    247174 non-null float64
favorites_11         247174 non-null float64
number_votes_11      247174 non-null float64
clicks_12            317040 non-null float64
comment_number_12    317040 non-null float64
favorites_12         317040 non-null float64
number_votes_12      317040 non-null float64
clicks_01            313540 non-null float64
comment_number_01    313540 non-null fl

In [22]:
# Rezeptsammlerin now only has 679 recipes instead of 870:
len(kochdf.loc[kochdf['user'] == "Rezeptsammlerin"]['name'])

679

In [23]:
len(kochdf['name'])

323895

In [24]:
# There is missing data again, so we drop them as well:
kochdf = kochdf.dropna()

In [25]:
# This has reduced the dataset by approx. a third:
len(kochdf['name'])

221649

In [26]:
# Export CSV
kochdf.to_csv('data/trend_data.csv')

In [27]:
# Aggregation: clicks
print("clicks:")
print(kochdf['clicks'].sum())
print(kochdf['clicks_11'].sum())
print(kochdf['clicks_12'].sum())
print(kochdf['clicks_01'].sum())
print(kochdf['clicks_02'].sum())
print(kochdf['clicks_03'].sum())

# Aggregation: comment_number
print("\n comment_number:")
print(kochdf['comment_number'].sum())
print(kochdf['comment_number_11'].sum())
print(kochdf['comment_number_12'].sum())
print(kochdf['comment_number_01'].sum())
print(kochdf['comment_number_02'].sum())
print(kochdf['comment_number_03'].sum())

# Aggregation: favorites
print("\n favorites:")
print(kochdf['favorites'].sum())
print(kochdf['favorites_11'].sum())
print(kochdf['favorites_12'].sum())
print(kochdf['favorites_01'].sum())
print(kochdf['favorites_02'].sum())
print(kochdf['favorites_03'].sum())

# Aggregation: number_votes
print("\n number_votes:")
print(kochdf['number_votes'].sum())
print(kochdf['number_votes_11'].sum())
print(kochdf['number_votes_12'].sum())
print(kochdf['number_votes_01'].sum())
print(kochdf['number_votes_02'].sum())
print(kochdf['number_votes_03'].sum())

clicks:
534432997
540453930.0
548861248.0
561406262.0
568512033.0
575692489.0

 comment_number:
1448716
1448859.0
1449724.0
1449908.0
1449994.0
1450034.0

 favorites:
1580470
1592317.0
1607046.0
1622601.0
1641426.0
1659256.0

 number_votes:
4187470
4187506.0
4189670.0
4190727.0
4190885.0
4192395.0


Now we actually only have the recipes left that have been in each snapshot of our the data. By dropping the duplicates and recipes with missing values, we have reduced the dataset's size significantly, but created a consistent set that is suitable for trend analysis.

### Calculating the differences

In addition to the dataset above, which shows the numerical values for each timestep, we can also create a dataset that holds the deltas (i.e. the differences between each of the timesteps). The following abbreviations are used:

- clicks = cl
- comment_number = co
- favorites = f
- number_votes = v
- delta(Okt-Nov) = d1
- delta(Nov-Dec) = d2
- delta(Dec-Jan) = d3
- delta(Jan-Feb) = d4
- delta(Feb-Mar) = d5

In [28]:
deltadf = pd.DataFrame(kochdf[['name', 'user']])

# Clicks
deltadf['cl_d1'] = kochdf['clicks_11']-kochdf['clicks']
deltadf['cl_d2'] = kochdf['clicks_12']-kochdf['clicks_11']
deltadf['cl_d3'] = kochdf['clicks_01']-kochdf['clicks_12']
deltadf['cl_d4'] = kochdf['clicks_02']-kochdf['clicks_01']
deltadf['cl_d5'] = kochdf['clicks_03']-kochdf['clicks_02']

# Comment_number
deltadf['co_d1'] = kochdf['comment_number_11']-kochdf['comment_number']
deltadf['co_d2'] = kochdf['comment_number_12']-kochdf['comment_number_11']
deltadf['co_d3'] = kochdf['comment_number_01']-kochdf['comment_number_12']
deltadf['co_d4'] = kochdf['comment_number_02']-kochdf['comment_number_01']
deltadf['co_d5'] = kochdf['comment_number_03']-kochdf['comment_number_02']

# Favorites
deltadf['f_d1'] = kochdf['favorites_11']-kochdf['favorites']
deltadf['f_d2'] = kochdf['favorites_12']-kochdf['favorites_11']
deltadf['f_d3'] = kochdf['favorites_01']-kochdf['favorites_12']
deltadf['f_d4'] = kochdf['favorites_02']-kochdf['favorites_01']
deltadf['f_d5'] = kochdf['favorites_03']-kochdf['favorites_02']

# Number_votes
deltadf['v_d1'] = kochdf['number_votes_11']-kochdf['number_votes']
deltadf['v_d2'] = kochdf['number_votes_12']-kochdf['number_votes_11']
deltadf['v_d3'] = kochdf['number_votes_01']-kochdf['number_votes_12']
deltadf['v_d4'] = kochdf['number_votes_02']-kochdf['number_votes_01']
deltadf['v_d5'] = kochdf['number_votes_03']-kochdf['number_votes_02']

deltadf.head(10)

,name,user,cl_d1,cl_d2,cl_d3,cl_d4,cl_d5,co_d1,co_d2,co_d3,...,f_d1,f_d2,f_d3,f_d4,f_d5,v_d1,v_d2,v_d3,v_d4,v_d5
0,Japanischer Soufflé-Käsekuchen,KB_Redaktion,7345.0,11231.0,10371.0,10253.0,11102.0,0.0,0.0,0.0,...,70.0,-1.0,2.0,87.0,7.0,0.0,0.0,0.0,0.0,0.0
1,"Chili con Carne ""Spezial""",mareikus,51100.0,70205.0,77668.0,61144.0,21439.0,0.0,0.0,0.0,...,12.0,29.0,31.0,28.0,21.0,0.0,0.0,2.0,0.0,1.0
2,Pfannkuchen Grundrezept,zaphi67,92144.0,133786.0,132484.0,165010.0,121558.0,0.0,0.0,0.0,...,6.0,7.0,6.0,8.0,9.0,1.0,0.0,0.0,0.0,0.0
3,Allerbester Käsekuchen♥,Schokoprinzessin,17948.0,21743.0,23374.0,25119.0,27446.0,0.0,0.0,0.0,...,11.0,20.0,11.0,17.0,17.0,1.0,2.0,1.0,0.0,0.0
4,Flammkuchen Schneller Flammkuchen ohne Hefe,KathiB,5581.0,4721.0,4366.0,3180.0,2649.0,0.0,0.0,0.0,...,8.0,3.0,5.0,5.0,7.0,0.0,0.0,0.0,0.0,1.0
5,Pizza-Schnecken,TimejaKijara,854.0,1042.0,1483.0,1277.0,1792.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Zucchini-Hackauflauf mit Frischkäse und Spätzle,romantica,1236.0,1338.0,1078.0,1268.0,1097.0,1.0,0.0,0.0,...,19.0,30.0,24.0,43.0,34.0,0.0,0.0,1.0,0.0,0.0
7,Amerikanische Pancakes,Eni0,3025.0,1965.0,1644.0,1438.0,2012.0,0.0,0.0,2.0,...,48.0,40.0,15.0,9.0,37.0,0.0,0.0,0.0,0.0,0.0
8,Allerbester Rhabarberkuchen♥,Schokoprinzessin,178.0,157.0,163.0,162.0,420.0,0.0,0.0,0.0,...,0.0,0.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
9,Schichtsalat,Kruemelmonster,5367.0,6447.0,20597.0,8684.0,7950.0,0.0,0.0,0.0,...,2.0,1.0,6.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0


In [32]:
for i, words in enumerate(kochdf['name']):
    for char in ['"','-','=','!','(',')','.','♥',',',':','~','„','“','/','–','&','+',';','*','☆']:
        words = words.replace(char,' ')
    words = words.replace('  ',' ')

In [33]:
# Export CSV
deltadf.to_csv('data/trend_deltas.csv')